In [37]:
import math

In [39]:
def substitutionSh(word, k, deSh = False, **kwargs):
    delta = -k if deSh else k #определяем отступ
    alphc = ord("Z") - ord("A") + 1 #размер алфавита
    res = ''
    for i in word:
        if not i.isalpha():
            res += i
            continue
        if i.islower(): #сдвигаем элемент по ascii таблице в зависимости от регистра
            res += chr((ord(i) + delta - ord("a")) % alphc + ord("a")) 
        else:
            res += chr((ord(i) + delta - ord("A")) % alphc + ord("A"))
    return res
alph = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
def getCaesarAlph(k, keyword, alph):
    keyword = keyword.upper()
    ln = len(alph) #длина алфавита
    usAlph = alph
    shAlph = [i for i in alph] #зашифрованный алфавит 
    indexes = [i for i in range(ln)] #индексы алфавита
    endind = 0 #индекс первого элемента после зашифрованного слова
    for i in range(len(keyword)): #заменяем буквы алфавита на буквы слова
        endind = (k+i) % ln 
        indexes.remove(endind)
        shAlph[endind] = keyword[i]
        usAlph = usAlph.replace(keyword[i], '')
    startind = indexes.index(endind+1)
    for i in range(len(indexes)): #заменяем оставльные буквы алфавита
        shAlph[(indexes[startind]+i) % ln] = usAlph[i]
    alph += alph.lower()
    shAlph += [i for i in map(lambda x: x.lower(), shAlph)]
    res = dict(zip([i for i in alph], shAlph)), dict(zip( shAlph, [i for i in alph]))

    return res #[0] -шифровка , [1] - дешифровка
def caeasarShifr(word, k, keyword, deSh = False, alph = alph, **kwargs):
    a = getCaesarAlph(k, keyword, alph)
    res = ''
    if deSh:
        for i in word:
            if i.isalpha():
                try:
                    res += a[1][i]
                except:
                    res += i
            else:
                res += i
    else:
        for i in word:
            if i.isalpha():
                try:
                    res += a[0][i]
                except:
                    res += i
            else:
                res += i
    return res

In [134]:
#Частота букв
ABC = "zqjxkvbwypgumcfldhsirnoate" #1-граммы
ABC = ['th', 'he', 'in', 'er', 'an', 're', 'on', 'at', 'en', 'nd', 'ti', 'es', 'or', 'te', 'of', 'ed', 'is', 'it', 'al', 'ar'] #2-граммы 
frequency = [0.07, 0.11, 0.13, 0.15, 0.4, 0.9, 1.4, 1.5, 1.9, 1.9, 2, 2.4, 2.5, 2.7, 2.9, 3.4, 3.8, 5.2, 6.1, 6.3, 6.8, 7.1, 7.9, 8.1, 10.5, 13] #1-граммы
frequency = [ #2-граммы
3.56,
3.07,
2.43,
2.05,
1.99,
1.85,
1.76,
1.49,
1.45,
1.35,
1.34,
1.34,
1.28,
1.20,
1.17,
1.17,
1.13,
1.12,
1.09,
1.07
]
freq = dict(zip(ABC, [round(i / 100, 4) for i in frequency])) #словарь соответствий

In [29]:
with open("text.txt") as f: #считываем текст из файла
    mainText = f.read()
#mainText = "By definition, IP is an estimate of the sum of the squares of the probabilities of each letter. With a multi-alphabetic substitution for the English language, depending on the number of alphabets used in encryption (m), the IP is expected"

In [141]:
def recognizeStateMachine(func): #декоратор - конечный автомат распознвания слов длины wordLen, func - функция применяемая к распознанному слову
    """
    funcOnNotAlpha - функция на обработку не буквенных символов
    """
    def f(string, wordLen, funcOnNotAlpha = lambda x: x): 
        word = ''
        for i in string:
            if i.isalpha():
                word += i.lower()
                if len(word) == wordLen:
                    word = func(word)
            else:
                funcOnNotAlpha(word + i)
                word = ''
    return f

In [ ]:
def alphaLen(string, wordLen = 1): #подсчет количества слов длины wordLen
    ln = [0]
    @recognizeStateMachine
    def f(word):
        ln[0] += 1
        return word[1:]
    f(string, wordLen)
    return ln[0]
def alphaCount(string, wordLen = 1): #подсчитываем количественные отношения всех слов длины wordLen к общему кол-ву слов
    res = dict({})
    ln = alphaLen(string, wordLen)
    @recognizeStateMachine
    def f(word):
        if word not in res:
            res.update({word : 1 / ln})
        else:
            res[word] += 1 / ln
        return word[1:]
    f(string, wordLen)
    return res

In [42]:
def dictSortByVal(dictt): #сортировка словаря по values
    return dict(sorted(dictt.items(), key= lambda x: x[1], reverse=True))

In [147]:
def sybmolDesh(string, wordLen = 1, freq = freq, markReplace = ''): #дешифровка частотным методом, markReplace - символ для выделения измененных символов
    freq = dictSortByVal(freq) #словарь введенных частот
    allpc = dictSortByVal(alphaCount(string, wordLen)) #словрь подсчитанных в зашифрованном тексте
    replaceDict = dict(zip(allpc.keys(), freq.keys())) # словарь соответвия
    res = ['']
    @recognizeStateMachine #заменяем все слова соответствующие словарю соответствия
    def f(word):
        if word in replaceDict:
            res[0] += markReplace +  replaceDict[word] + markReplace
            return ''
        else:
            res[0] += word[0]
            return word[1:]
    def isNotAlpha(word):
        res[0] += word
    f(string, wordLen, isNotAlpha)
    return res[0]

In [40]:
#print(mainText, "\n")
print(caeasarShifr(mainText, 2, "keyword"), "\n")
#print(sybmolDesh(substitutionSh(mainText, 2), 2), "\n")

Xehcw Rdncyl sxm xg Xpmnldxg-zhlg Oylfxg ihcdndkdxg srh sxm nry edknxnhl hw Oylfxgu wlhf 1933 pgndc rdm eyxnr dg 1945. Ry lhmy nh ihsyl xm nry cyxeyl hw nry Gxvd Ixlnu,[x] zykhfdgo nry krxgkycchl dg 1933 xge nryg xmmpfdgo nry ndncy hw WГјrlyl pge Lydkrmbxgvcyl dg 1934.[z] Epldgo rdm edknxnhlmrdi, ry dgdndxnye Shlce Sxl DD dg Yplhiy zu dgqxedgo Ihcxge hg 1 Myinyfzyl 1939. Ry sxm kchmycu dgqhcqye dg fdcdnxlu hiylxndhgm nrlhporhpn nry sxl xge sxm kygnlxc nh nry iyliynlxndhg hw nry Rhchkxpmn, nry oyghkdey hw xzhpn mdt fdccdhg Aysm xge fdccdhgm hw hnryl qdkndfm.

Rdncyl sxm zhlg dg Xpmnldx-Rpgoxlu xge sxm lxdmye gyxl Cdgv. Ry cdqye dg Qdyggx cxnyl dg nry wdlmn eykxey hw nry 1900m xge fhqye nh Oylfxgu dg 1913. Ry sxm eykhlxnye epldgo rdm mylqdky dg nry Oylfxg Xlfu dg Shlce Sxl D. Dg 1919, ry ahdgye nry Oylfxg Shlbylm' Ixlnu (EXI), nry ilykplmhl hw nry Gxvd Ixlnu, xge sxm xiihdgnye cyxeyl hw nry Gxvd Ixlnu dg 1921. Dg 1923, ry xnnyfinye nh mydvy ohqylgfygnxc ihsyl dg x wxdcye khpi dg Fpgdkr x